<a href="https://colab.research.google.com/github/phylars/ILEE_CSK/blob/ipynb/ilee_2d_mode_with_custom_effective_area.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Preparation**

Please transfer you confocal format files to raw (stack) tiff images using ImageJ/FIJI, and put them in a new folder.
*   If you wish to run the pipeline in Google Colab, please upload the tiff folder to your Google Drive


In [ ]:
#@title Mount your google drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title Install ILEE_CSK and dependencies
!pip install -U ILEE_CSK
import ILEE_CSK
import skimage.io as io
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scikit_posthocs as sp

In [3]:
#@title Input the image folder containing all the raw tiffs in this experiment

folder_path = ''  #@param {type: "string"}

In [ ]:
#@title Estimate optimal K2
#@markdown K2 is an parameter required for ILEE processing. We recommend use the optimal K2 calculated by this function for most of the cases.

#@markdown According to the projected images of each channel, shown above, please input channel index of cytoskeleton fluorescence:

objective_channel_index = 0  #@param {type: "number"}
optimal_K2 = ILEE_CSK.opt_k2(folder_path, target_channel = objective_channel_index)
print('The estimated optimal K2 is', optimal_K2)

# **Start processing your samples**

In [ ]:
#@title Set input parameters and run

#@markdown Input the path of your area excel file.
effective_areas_excel_file_path = ""  #@param {type: "string"}
#@markdown Set the channel for analysis.
objective_channel_index = 0  #@param {type: "number"}
#@markdown Set K1. Highly recommended set K1 to 2.5, unless you know what it means.
K1 = 2.5  #@param {type: "number"}
#@markdown Copy your optimal K2 to this place:
K2 = None  #@param {type: "number"}
#@markdown Set your Laplacian Matrix type, highly recommended use pL_8, unless you know what this means:
pL_type = "pL_8"  #@param ['pL_8', 'pL_4']

#@markdown Set pixel unit size by μm. Check the pixel size using ImageJ/Fiji or your confocal program. If set to 1, the length unit PU will be "pixel". If set to the true pixel size, the length unit PU will be "μm". This must be EXACTLY the SAME as the area data you input.
pixel_size = 1  #@param {type: "number"}


df_areas = pd.read_excel(effective_areas_excel_file_path)
areas = np.array(df_areas.area).astype('float')

df_result = ILEE_CSK.analyze_document_2D (folder_path = folder_path,
                                          obj_channel = objective_channel_index,
                                          k1 = K1,
                                          k2 = K2,
                                          pixel_size = 1,
                                          exclude_true_blank = True,
                                          effective_area_data = areas)





# **Get the result**

In [ ]:
#@title Show your data table
df_result.style

In [ ]:
#@title Download the data as Excel file.
from google.colab import files
df_result.to_excel('result.xlsx')
files.download('result.xlsx')

In [ ]:
#@title Visualize your data
#@markdown As noted above, to enable visualization and significance test, your tiff files must be named as "xxx(group)-xxx(index)".
ILEE_CSK.visualize_results(df_result)

In [ ]:
#@title Significant test

p_thres = 0.05  #@param {type: "number"}

Data_property = "I am measuring biological samples and/or want to learn the biological features at my experiment condition. (Use post-hoc, FDR Benjamini\u2013Hochberg method)"  #@param ['I am measuring biological samples and/or want to learn the biological features at my experiment condition. (Use post-hoc, Holm–Bonferroni method)', 'I am measuring biological samples and/or want to learn the biological features at my experiment condition. (Use post-hoc, FDR Benjamini–Hochberg method)', 'I am not using data to interpret biological features and the image per-se is ground truth. (No post-hoc)']
if (Data_property == 'I am measuring biological samples and/or want to learn the biological features at my experiment condition. (Use post-hoc, Holm–Bonferroni method)'):
  p_adjust = 'holm'
elif (Data_property == 'I am measuring biological samples and/or want to learn the biological features at my experiment condition. (Use post-hoc, FDR Benjamini–Hochberg method)'):
  p_adjust = 'fdr_bh'
else:
  p_adjust = None
#@markdown Recommend Holm-Bonferroni method for post-hoc. In some rare cases, Holm-Bonferroni may have very abnormal results; try FDR Benjamini–Hochberg method instead then.

print('Testing with P value threshold at', p_thres)
print('Family-wise error correction :', p_adjust)
print('\n')

sd_table = ILEE_CSK.total_df_multiple_comparison (df = df_result, cat_var_column = 'group', p_thres = p_thres, column_types = ['float'], p_adjust = p_adjust)
sd_table.style